In [76]:
import os, sys

import pathlib
import pandas as pd
import numpy as np
from easydict import EasyDict as edict
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import time

dir_path = pathlib.Path().absolute()
file = '../data/imputed_data.csv'
data_path = os.path.join(dir_path, file)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv(data_path  ,index_col=False).drop(['Unnamed: 0'], axis=1)
df.reset_index(drop=True, inplace=True)
assert df.isnull().sum().sum() == 0

df.drop(columns=['SPI_X1','SPI_Y1','SPI_X2','SPI_Y2','SPI_L1','SPI_W1','SPI_L2','SPI_W2'], inplace=True)
df.head(10)

,CRD,PartType,Orient.,X,Y,SPI_X_AVG,SPI_Y_AVG,SPI_L,SPI_W,PRE_X,PRE_Y,PRE_A,PRE_L,PRE_W,POST_X,POST_Y,POST_A,POST_L,POST_W,Job,Chip_L,Chip_W,SPI_VOLUME_MEAN
0,B1,R0402,90.0,31.26,100.11,-52.00,61.90,61.90,52.00,-19.887293,-1.840963,-1.877,-1.840963,19.887293,-7.440417,28.108231,1.613,28.108231,7.440417,CENTER,400.0,200.0,78.71525
1,B2,R0402,90.0,31.83,100.11,-47.00,63.70,63.70,47.00,-26.766272,6.898049,-1.449,6.898049,26.766272,5.487133,22.242701,0.000,22.242701,-5.487133,CENTER,400.0,200.0,77.33190
2,B3,R0402,90.0,32.40,100.11,-41.60,72.35,72.35,41.60,-32.058273,8.125244,-3.278,8.125244,32.058273,-19.644203,12.971593,1.552,12.971593,19.644203,CENTER,400.0,200.0,77.17380
3,B4,R0402,90.0,32.97,100.11,-50.60,63.10,63.10,50.60,-23.437572,-6.195457,-3.337,-6.195457,23.437572,-22.517658,26.396389,0.000,26.396389,22.517658,CENTER,400.0,200.0,77.96495
4,B5,R0402,90.0,33.54,100.11,-50.00,69.45,69.45,50.00,-5.187286,7.085744,-3.408,7.085744,5.187286,-21.413681,29.193306,3.611,29.193306,21.413681,CENTER,400.0,200.0,75.37700
5,B6,R0402,90.0,31.26,99.20,-49.10,59.75,59.75,49.10,-31.059980,1.531298,3.611,1.531298,31.059980,-5.925862,17.682121,0.000,17.682121,5.925862,CENTER,400.0,200.0,77.66230
6,B7,R0402,90.0,31.83,99.20,-44.95,60.10,60.10,44.95,-20.728939,10.258562,-3.247,10.258562,20.728939,-17.024027,33.615910,3.455,33.615910,17.024027,CENTER,400.0,200.0,77.56575
7,B8,R0402,90.0,32.40,99.20,-39.05,70.00,70.00,39.05,-27.195978,11.357373,-2.031,11.357373,27.195978,-1.401807,29.627155,3.337,29.627155,1.401807,CENTER,400.0,200.0,79.10090
8,B9,R0402,90.0,32.97,99.20,-46.55,62.45,62.45,46.55,-19.822590,1.057152,-3.047,1.057152,19.822590,-20.989681,15.195607,3.310,15.195607,20.989681,CENTER,400.0,200.0,81.96445
9,B10,R0402,90.0,33.54,99.20,-53.25,71.15,71.15,53.25,-26.842043,3.604794,-3.022,3.604794,26.842043,-17.169552,31.046926,-1.660,31.046926,17.169552,CENTER,400.0,200.0,79.36680


In [77]:
# switch 90 data to 0 data
def switchOrient(x90, y90):
    y0 = float(x90)
    x0 = float(-y90)
    return x0, y0 

In [78]:
# 전체 SPI offset 꽈 post offset의 벡터거리 차이에 대한 pre/post angle관계 

# 1. group by chip type
ABSOLUTE_ANGLE = True
BY_CHIP_PERCENTAGE = True
PRE_OR_POST_ANGLE = 'pre'
DISTANCE = 'spi' # [DISTANCE] - POST 
STOP_AFTER = 10 # rows

dfgroups = df.groupby(['PartType']) 
# df_chips :: holds dataframes [diff_distance, angle]
df_chips = edict()
# df_chips_condition :: holds dataframes by condition (2)
#   (0: condition 1 [SPI<=CHIP]
#    1: condition 2 [SPI>CHIP])
df_chips_condition = edict()
# df_chips_cond_quad :: holds dataframes by quadrant condition (4) for SPI and Chip (<= or >) (2) -> (8 total)
df_chips_cond_quad = edict()
# df_chips_condition2 :: holds dataframes by the following conditions
#   (0: SPI (+,+) and chip (+,+) and SPI <= chip, 
#    1: SPI (+,+) and chip (-,-) and SPI <= chip, 
#    2: SPI (+,+) and chip (-,-) and SPI > chip, 
#    3: SPI (-,+) and chip (-.+) and SPI <= chip, 
#    4: SPI (-,+) and chip (+,+) and SPI <= chip, 
#    5: SPI (-,+) and chip (-,-) and SPI > chip, 
df_chips_condition2 = edict() # Chip이 SPI 와 동일한 뱡향으로 이동한경우, 반대방향으로 이동한경우 (6)


In [79]:

start = time.time()

for _, (chiptype, group) in enumerate(dfgroups):
    maxval = 0
    print(chiptype,'...')
    time.sleep(0.3)
    
    if ABSOLUTE_ANGLE == True:
        group['PRE_A'] = group['PRE_A'].abs()
        group['POST_A'] = group['POST_A'].abs()
    
    # initialize dictionaries of each size
    df_chips[f'{chiptype}'] = pd.DataFrame(columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post'])
    df_chips_condition[f'{chiptype}'] = \
            [pd.DataFrame(columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post']) for i in range(2)]
    df_chips_cond_quad[f'{chiptype}'] = \
            [pd.DataFrame(columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post']) for i in range(8)]
    df_chips_condition2[f'{chiptype}'] = \
            [pd.DataFrame(columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post']) for i in range(6)]
    
    chiplength = group['Chip_L'].values[0]
    
    # iterate through each row and compute offset distance difference
    t = tqdm(group.iterrows(), total=len(group))
    for idx, row in t:
        t.set_description(f'Row: {idx}')
        
        spi_x_avg, spi_y_avg = row[['SPI_X_AVG','SPI_Y_AVG']]
        pad_center_x, pad_center_y = row[['X','Y']] * 1000 # mm --> um change of units
        spi_volume_mean = row['SPI_VOLUME_MEAN']
        pre_x, pre_y = row[['PRE_X','PRE_Y']]
        post_x, post_y = row[['POST_X','POST_Y']]
        pre_angle, post_angle = row[['PRE_A','POST_A']]
        orientation = row['Orient.']
        
        ''' rotate 90 to 0 '''
        if orientation == 90:
            spi_x_avg, spi_y_avg = switchOrient(spi_x_avg, spi_y_avg)
            pad_center_x, pad_center_y = switchOrient(pad_center_x, pad_center_y)
            pre_x, pre_y = switchOrient(pre_x, pre_y)
            post_x, post_y = switchOrient(pre_x, pre_y)
        
        ''' compute vector norms for comparison '''
        spi_norm = np.linalg.norm((spi_x_avg, spi_y_avg))
        post_norm = np.linalg.norm((post_x, post_y))
        
        ''' 
        compute distance between two offset vectors from the origin (칩의 좌측하단)
        and append to df_temp dataframe
        '''
        # vector offset distance difference from SPI
        xval, yval = 0, 0
        if DISTANCE == 'spi':
            xval = spi_x_avg - post_x
            yval = spi_y_avg - post_y
        # elif DISTANCE == 'pad':
        #     xval = pad_center_x - post_x
        #     yval = pad_center_y - post_y
        diff_distance = np.linalg.norm((xval, yval))
        
        if BY_CHIP_PERCENTAGE == True:
            diff_distance = diff_distance / chiplength * 100
        
        if maxval < diff_distance:
            maxval = diff_distance
        
        angle = 0
        if PRE_OR_POST_ANGLE == 'pre':
            angle = pre_angle
        elif PRE_OR_POST_ANGLE == 'post':
            angle = post_angle
        
        # vector offset difference from PRE
        dist_pre = np.linalg.norm((pre_x, pre_y))
        dist_post = np.linalg.norm((post_x, post_y))
        
        # value to add
        values_to_add = [diff_distance, angle, dist_pre, dist_post]
        
        ''' append distance and angle (all) '''
        df_chips[f'{chiptype}'].loc[len(df_chips[f'{chiptype}']),:] = values_to_add
        
        ''' append distance and angle by condition only '''
        if spi_norm <= post_norm:
            df_chips_condition[f'{chiptype}'][0].loc[len(df_chips_condition[f'{chiptype}'][0]),:] = \
                values_to_add
        else:
            df_chips_condition[f'{chiptype}'][1].loc[len(df_chips_condition[f'{chiptype}'][1]),:] = \
                values_to_add
        
        ''' append distance and angle by condition and quadrant for SPI and Post (Chip) '''
        # df_chips_cond_quad
        case = -1
        # case 1: SPI offset is (+, +), Post offset is (+,+), SPI offset <= Post offset
        if spi_x_avg >= 0 and spi_y_avg >= 0 and post_x >= 0 and post_y >= 0 \
            and spi_norm <= post_norm:
            case = 0
        # case 2: SPI offset is (+, +), Post offset is (+,+), SPI offset > Post offset
        elif spi_x_avg >= 0 and spi_y_avg >= 0 and post_x >= 0 and post_y >= 0 \
            and spi_norm > post_norm:
            case = 1
        # case 3: SPI offset is (-, +), Post offset is (-,+), SPI offset <= Post offset
        elif spi_x_avg <= 0 and spi_y_avg >= 0 and post_x <= 0 and post_y >= 0 \
            and spi_norm <= post_norm:
            case = 2
        # case 4: SPI offset is (-, +), Post offset is (-,+), SPI offset  > Post offset
        elif spi_x_avg <= 0 and spi_y_avg >= 0 and post_x <= 0 and post_y >= 0 \
            and spi_norm > post_norm:
            case = 3
        # case 5: SPI offset is (-, -), Post offset is (-, -), SPI offset <= Post offset
        elif spi_x_avg <= 0 and spi_y_avg <= 0 and post_x <= 0 and post_y <= 0 \
            and spi_norm <= post_norm:
            case = 4
        # case 6: SPI offset is (-, -), Post offset is (-, -), SPI offset > Post offset
        elif spi_x_avg <= 0 and spi_y_avg <= 0 and post_x <= 0 and post_y <= 0 \
            and spi_norm > post_norm:
            case = 5
        # case 7: SPI offset is (+, -), Post offset is (+,-), SPI offset <= Post offset
        elif spi_x_avg >= 0 and spi_y_avg <= 0 and post_x >= 0 and post_y <= 0 \
            and spi_norm <= post_norm:
            case = 6
        # case 8: SPI offset is (+, -), Post offset is (+,-), SPI offset > Post offset
        elif spi_x_avg >= 0 and spi_y_avg <= 0 and post_x >= 0 and post_y <= 0 \
            and spi_norm > post_norm:
            case = 7
        df_chips_cond_quad[f'{chiptype}'][case].loc[len(df_chips_cond_quad[f'{chiptype}'][case]),:] = \
            values_to_add
        
        ''' append distance and angle by alignment direction for SPI and Post (Chip) '''
        # df_chips_condition2
        # case 1: SPI offset is (+, +), Post offset is (+,+), SPI offset <= Post offset
        if spi_x_avg >= 0 and spi_y_avg > 0 and post_x >= 0 and post_y >= 0 \
            and spi_norm <= post_norm:
            case = 0
        # case 2a: SPI offset is (-, -), Post offset is (-, -), SPI offset <= Post offset
        elif spi_x_avg <= 0 and spi_y_avg <= 0 and post_x <= 0 and post_y <= 0 \
            and spi_norm <= post_norm:
            case = 1
        # case 2b: SPI offset is (-, -), Post offset is (-, -), SPI offset > Post offset
        elif spi_x_avg <= 0 and spi_y_avg <= 0 and post_x <= 0 and post_y <= 0 \
            and spi_norm > post_norm:
            case = 2  
        # case 3: SPI offset is (-, +), Post offset is (-,+), SPI offset <= Post offset      
        elif spi_x_avg <= 0 and spi_y_avg >= 0 and post_x <= 0 and post_y >= 0 \
            and spi_norm <= post_norm:
            case = 3
        # case 4a: SPI offset is (+, -), Post offset is (+,-), SPI offset <= Post offset
        elif spi_x_avg >= 0 and spi_y_avg <= 0 and post_x >= 0 and post_y <= 0 \
            and spi_norm <= post_norm:
            case = 4
        # case 4b: SPI offset is (+, -), Post offset is (+,-), SPI offset > Post offset
        elif spi_x_avg >= 0 and spi_y_avg <= 0 and post_x >= 0 and post_y <= 0 \
            and spi_norm > post_norm:
            case = 5
        df_chips_condition2[f'{chiptype}'][case].loc[len(df_chips_condition2[f'{chiptype}'][case]),:] = \
            values_to_add
            
        # if idx > 1000: #STOP_AFTER:
        #     break
    
    # sort by distance
    tmp = df_chips[f'{chiptype}'].to_numpy()
    df_chips[f'{chiptype}'] = tmp[tmp[:,0].argsort()]    
    
    tcondition = trange(2)
    for condition in tcondition:
        tcondition.set_description('sorting condition dataframes by distance SPI:')
        tmp = df_chips_condition[f'{chiptype}'][condition].to_numpy()
        df_chips_condition[f'{chiptype}'][condition] = tmp[tmp[:,0].argsort()]    
        
    tcondition = trange(6)
    for condition in tcondition:
        tcondition.set_description('sorting condition 2 dataframes by distance SPI:')
        tmp = df_chips_condition2[f'{chiptype}'][condition].to_numpy()
        df_chips_condition2[f'{chiptype}'][condition] = tmp[tmp[:,0].argsort()]    
    
    tcase = trange(8)
    for case in tcase:
        tcase.set_description('sorting case dataframes by distance SPI:')
        tmp = df_chips_cond_quad[f'{chiptype}'][case].to_numpy()
        df_chips_cond_quad[f'{chiptype}'][case] = tmp[tmp[:,0].argsort()]    
    
print(f'It took {time.time() - start} seconds')

R0402 ...
R1005 68.29237813803111
R0603 ...
R1005 60.64582510055827
R1005 ...


Row: 100764:  55%|█████▍    | 33255/60750 [03:29<04:18, 106.28it/s]

In [32]:
pd.DataFrame(df_chips['R0402'],dtype=float).iloc[:,0].describe()

count    751.000000
mean      13.955640
std        6.060263
min        0.537711
25%        9.984524
50%       13.588446
75%       18.701015
max       32.198364
Name: 0, dtype: float64

In [34]:
# save files as pickle formats
try:
    import pickle5 as pickle
except ImportError:  # Python 3.x
    import pickle
with open('pickle/df_chips.p', 'wb') as fp:
    pickle.dump(df_chips, fp, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickle/df_chips_condition.p', 'wb') as fp:
    pickle.dump(df_chips_condition, fp, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickle/df_chips_condition2.p', 'wb') as fp:
    pickle.dump(df_chips_condition2, fp, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickle/df_chips_cond_quad.p', 'wb') as fp:
    pickle.dump(df_chips_cond_quad, fp, protocol=pickle.HIGHEST_PROTOCOL)

# JSON
# import json
# with open('data.json', 'w') as fp:
#     json.dump(data, fp)
# with open('data.json', 'r') as fp:
#     data = json.load(fp)

In [7]:
# load pickle
import pickle5 as pickle
with open('pickle/df_chips.p', 'rb') as fp:
    df_chips = pickle.load(fp)
with open('pickle/df_chips_condition.p', 'rb') as fp:
    df_chips_condition = pickle.load(fp)
with open('pickle/df_chips_condition2.p', 'rb') as fp:
    df_chips_condition2 = pickle.load(fp)
with open('pickle/df_chips_cond_quad.p', 'rb') as fp:
    df_chips_cond_quad = pickle.load(fp)

chips = ['R0402','R0603','R1005']
for chip in chips:
    df_chips[chip] = pd.DataFrame(df_chips[chip],dtype=float,columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post'])
    for condition1 in range(2):
        df_chips_condition[chip][condition1] = pd.DataFrame(df_chips_condition[chip][condition1], dtype=float, columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post'])
    for condition2 in range(6):
        df_chips_condition2[chip][condition2] = pd.DataFrame(df_chips_condition2[chip][condition2], dtype=float, columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post'])
    for case in range(8):
        df_chips_cond_quad[chip][case] = pd.DataFrame(df_chips_cond_quad[chip][case], dtype=float, columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post'])
df_chips['R0402']['dist_SPI'].head()


0    0.085336
1    0.095069
2    0.131807
3    0.132821
4    0.134384
Name: dist_SPI, dtype: float64

In [12]:
# visualize
chiplengths = edict()
for _, (chiptype, group) in enumerate(dfgroups):
    chiplengths[chiptype] = group['Chip_L'].values[0]

chips = ['R0402', 'R0603', 'R1005']
for chip in chips:
    print(chip)
    for case in range(8):
        print(f"case {case}: {len(df_chips_cond_quad[chip][case])}")
        df_chips_cond_quad[chip][case] = pd.DataFrame(df_chips_cond_quad[chip][case], 
                                                      columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post'])
    print()   
    for condition in range(2):
        print(f"condition {condition}: {len(df_chips_condition[chip][condition])}")
        df_chips_condition[chip][condition] = pd.DataFrame(df_chips_condition[chip][condition], 
                                                           columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post'])
    print()   
    for condition in range(6):
        print(f"condition2 {condition}: {len(df_chips_condition2[chip][condition])}")
        df_chips_condition2[chip][condition] = pd.DataFrame(df_chips_condition2[chip][condition], 
                                                           columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post'])
    print()
    print(f"Total: {len(df_chips[chip])}")
    df_chips[chip] = pd.DataFrame(df_chips[chip], columns=['dist_SPI',f'{PRE_OR_POST_ANGLE}_angle','dist_pre','dist_post'])
    if BY_CHIP_PERCENTAGE == True:
        df_chips[chip]['dist_SPI'] /= chiplengths[chip]
        df_chips[chip]['dist_SPI'] *= 100
    print('='*67)

R0402
case 0: 1398
0.002099755468961616 400.0
0.000524938867240404
case 1: 7286
0.0034324340005440533 400.0
0.0008581085001360133
case 2: 398
0.001485450111647962 400.0
0.0003713625279119905
case 3: 5317
0.005672145137932784 400.0
0.001418036284483196
case 4: 545
0.001333382311890967 400.0
0.00033334557797274174
case 5: 3057
0.0020594908987697615 400.0
0.0005148727246924404
case 6: 1581
0.0033135205998169716 400.0
0.0008283801499542429
case 7: 41168
0.0031820015552955746 400.0
0.0007955003888238936

condition 0: 14793
condition 1: 45957

condition2 0: 1398
condition2 1: 7831
condition2 2: 3057
condition2 3: 5715
condition2 4: 1581
condition2 5: 41168

Total: 60750
R0603
case 0: 853
0.0012182099207186659 600.0
0.0002030349867864443
case 1: 4177
0.0009437636894532869 600.0
0.00015729394824221449
case 2: 553
0.0010734131027412901 600.0
0.00017890218379021502
case 3: 5576
1.3291854079432161e-05 600.0
2.2153090132386932e-06
case 4: 1015
0.0005498864871293413 600.0
9.164774785489021e-05
case

In [10]:
chip = 'R0402'
condition = 0
case = 0
df_chips_cond_quad[chip][case].astype(float).describe()

,dist_SPI,pre_angle,dist_pre,dist_post
count,1398.000000,1398.000000,1398.000000,1398.000000
mean,0.495333,1.680706,56.299577,52.056255
std,0.329849,1.515669,19.189428,20.601458
min,0.008399,0.000000,10.423906,4.637021
25%,0.236921,0.000000,42.620293,37.166021
50%,0.420955,1.539500,54.933813,50.722919
75%,0.706670,3.278000,69.211330,66.654800
max,1.912190,22.381001,136.464221,136.464221


In [13]:
img_path = './img'
if not os.path.exists(img_path):
    os.makedirs(img_path)
os.chdir(img_path)

In [60]:
''' quadrant '''
# split into distance groups (15) and compute mean and variance
chips = ['R0402', 'R0603', 'R1005']
cases = range(8)
conditions = range(2)
conditions2 = range(6)

stat_data = edict()
post_pre = edict()
for chip in chips:
    stat_data[chip] = edict()
    post_pre[chip] = edict()
    for case in cases:
        
        data = df_chips_cond_quad[chip][case].astype(float).copy()
        post_pre[chip][str(case)] = pd.DataFrame(data.iloc[:, 2:4], columns=['dist_pre','dist_post'])
        
        num_groups = 15
        data_range = data.iloc[:,0].max() - data.iloc[:,0].min()
        data_max_dists = [i * data_range / num_groups for i in np.arange(num_groups)]
        stat_data[chip][str(case)] = pd.DataFrame(columns=['dist_mean', 'angle_mean', 'angle_stddev'])
        for i, (data_max) in enumerate(data_max_dists[1:]):
            curr_i, next_i = i, i+1
            if next_i == num_groups:
                next_i = num_groups-1
            current_distances = data.iloc[:,0]
            data_valid_df = current_distances[(current_distances < data_max_dists[next_i]) & (current_distances > data_max_dists[curr_i])]
            data_valid_idx = data_valid_df.index
            data_group_df = data.iloc[data_valid_idx, :]

            if len(data_valid_idx) == 0 and curr_i != next_i:
                print('[INFO] at i:',i+1, '\t (',data_max_dists[curr_i],':',data_max_dists[next_i],'):', 'None')
                continue
            else:
                angle_mean = np.mean(data_group_df.iloc[:, 1])
                angle_stddev = np.std(data_group_df.iloc[:, 1])
                dist_median = np.mean([data_group_df.iloc[:,0].max(), data_group_df.iloc[:,0].min()])        
                if np.isnan(angle_mean) or np.isnan(angle_stddev) or np.isnan(dist_median):
                    angle_mean, angle_stddev, dist_median = 0, 0, 0
#                     print(f'>> at offset distance difference {data_max}: empty (no sample)')
            stat_data[chip][str(case)].loc[len(stat_data[chip][str(case)]),:] = [dist_median, angle_mean, angle_stddev]
# print(chip, case, '\n', stat_data, end='\n\n')

[INFO] at i: 11 	 ( 0.4758198786223906 : 0.5234018664846297 ): None
[INFO] at i: 13 	 ( 0.5709838543468687 : 0.6185658422091078 ): None
[INFO] at i: 14 	 ( 0.6185658422091078 : 0.6661478300713469 ): None
[INFO] at i: 14 	 ( 0.2014246506513799 : 0.21691885454763993 ): None


In [61]:
# visualize angle vs dist
colordict = ['r','b']

for j, chip in enumerate(chips):
    fig = plt.figure(figsize=(15,20))
    fig_pre_post = plt.figure(figsize=(6, 6))
    for i, case in enumerate(cases):
        ax = fig.add_subplot(len(cases)/2, 2, i+1)
        stat_data[chip][str(case)].plot(ax=ax, kind='scatter',x='dist_mean', y='angle_mean', color=colordict[0], label=f'{chip}_mean')
        stat_data[chip][str(case)].plot(ax=ax, kind='scatter',x='dist_mean', y='angle_stddev', color=colordict[1], label=f'{chip}_stddev')
#         print(stat_data[chip][str(case)])
        lower = [(m-s) for m,s in zip(stat_data[chip][str(case)].iloc[:,1], stat_data[chip][str(case)].iloc[:,2])]
        upper = [(m+s) for m,s in zip(stat_data[chip][str(case)].iloc[:,1], stat_data[chip][str(case)].iloc[:,2])]
        ax.fill_between(stat_data[chip][str(case)].iloc[:,0].astype(float), lower, upper, facecolor='green', alpha=0.2)
        
        ax.set_title(f'{chip}: {PRE_OR_POST_ANGLE} Angle vs. SPI-Post-AOI offset distance (case:{case+1})')
        if ABSOLUTE_ANGLE == True:
            ax.set_xlabel('SPI-Post-AOI offset distance in percentage of chip length (%)')
        else:
            ax.set_xlabel('SPI-Post-AOI offset distance (\u03BCm)')
        ax.set_ylabel(f'{PRE_OR_POST_ANGLE} angle (degree)')
    #     ax.set_xlim([np.min(pad_plot_data.iloc[:,0]), np.max(pad_plot_data.iloc[:,0])])
        # ax.set_ylim([0, 3.5])
        ax.legend()
        ax.grid()
        
        ax_pre_post = fig_pre_post.add_subplot(len(cases)/2, 2, i+1)    
        dist_pre = post_pre[chip][str(case)]['dist_pre']
        dist_post = post_pre[chip][str(case)]['dist_post']
        ax_pre_post.scatter(dist_pre, dist_post, alpha=0.3, label=f'{case+1}')
        ax_pre_post.set_xlabel('Pre-AOI offset')
        ax_pre_post.set_ylabel('Post-AOI offset')
        ax_pre_post.set_title(f'{chip}_{case+1}_pre_post')
        ax_pre_post.legend()
        ax_pre_post.grid()
    
    fig_pre_post.tight_layout()
    fig_pre_post.savefig(f'{chip}_pre_post_cases.png')
    fig_pre_post.clf()
        
    fig.tight_layout()
    fig.savefig(f'{chip}_cases.png')
    fig.clf()



<Figure size 1080x1440 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 1080x1440 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 1080x1440 with 0 Axes>

<Figure size 432x432 with 0 Axes>

In [62]:
# split into distance groups (15) and compute mean and variance
chips = ['R0402', 'R0603', 'R1005']
cases = range(8)
conditions = range(2)
conditions2 = range(6)

stat_data = edict()
post_pre = edict()
for chip in chips:
    stat_data[chip] = edict()
    post_pre[chip] = edict()
    for condition in conditions:        
        data = df_chips_condition[chip][condition].astype(float).copy()
        post_pre[chip][str(case)] = pd.DataFrame(data.iloc[:, 2:4], columns=['dist_pre','dist_post'])
        
        num_groups = 15
        data_range = data.iloc[:,0].max() - data.iloc[:,0].min()
        data_max_dists = [i * data_range / num_groups for i in np.arange(num_groups)]
        stat_data[chip][str(condition)] = pd.DataFrame(columns=['dist_mean', 'angle_mean', 'angle_stddev'])
        for i, (data_max) in enumerate(data_max_dists[1:]):
            curr_i, next_i = i, i+1
            if next_i == num_groups:
                next_i = num_groups-1
            current_distances = data.iloc[:,0]
            data_valid_df = current_distances[(current_distances < data_max_dists[next_i]) & (current_distances > data_max_dists[curr_i])]
            data_valid_idx = data_valid_df.index
            data_group_df = data.iloc[data_valid_idx, :]

            if len(data_valid_idx) == 0 and curr_i != next_i:
                print('[INFO] at i:',i+1, '\t (',data_max_dists[curr_i],':',data_max_dists[next_i],'):', 'None')
                continue
            else:
                angle_mean = np.mean(data_group_df.iloc[:, 1])
                angle_stddev = np.std(data_group_df.iloc[:, 1])
                dist_median = np.mean([data_group_df.iloc[:,0].max(), data_group_df.iloc[:,0].min()])        
                if np.isnan(angle_mean) or np.isnan(angle_stddev) or np.isnan(dist_median):
                    angle_mean, angle_stddev, dist_median = 0, 0, 0
            stat_data[chip][str(condition)].loc[len(stat_data[chip][str(condition)]),:] = [dist_median, angle_mean, angle_stddev]
# print(chip, case, '\n', stat_data, end='\n\n')

In [63]:
# visualize angle vs dist
colordict = ['r','b']
for j, chip in enumerate(chips):
    fig = plt.figure(figsize=(15,5))  
    fig_pre_post = plt.figure(figsize=(6, 6))
    for i, condition in enumerate(conditions):
        ax = fig.add_subplot(1,len(conditions),i+1)
        stat_data[chip][str(condition)].plot(ax=ax, kind='scatter',x='dist_mean', y='angle_mean', color=colordict[0], label=f'{chip}_mean')
        stat_data[chip][str(condition)].plot(ax=ax, kind='scatter',x='dist_mean', y='angle_stddev', color=colordict[1], label=f'{chip}_stddev')

        lower = [(m-s) for m,s in zip(stat_data[chip][str(condition)].iloc[:,1], stat_data[chip][str(condition)].iloc[:,2])]
        upper = [(m+s) for m,s in zip(stat_data[chip][str(condition)].iloc[:,1], stat_data[chip][str(condition)].iloc[:,2])]
        ax.fill_between(stat_data[chip][str(condition)].iloc[:,0].astype(float), lower, upper, facecolor='green', alpha=0.2)
        
        ax.set_title(f'{chip}: {PRE_OR_POST_ANGLE} Angle vs. SPI-Post-AOI offset distance (condition:{condition+1})')
        if ABSOLUTE_ANGLE == True:
            ax.set_xlabel('SPI-Post-AOI offset distance in percentage of chip length (%)')
        else:
            ax.set_xlabel('SPI-Post-AOI offset distance (\u03BCm)')
        ax.set_ylabel(f'{PRE_OR_POST_ANGLE} angle (degree)')
        # ax.set_ylim([0, 3.5])
        ax.legend()
        ax.grid()
        
                
        ax_pre_post = fig_pre_post.add_subplot(1, len(conditions), i+1)    
        dist_pre = post_pre[chip][str(case)]['dist_pre']
        dist_post = post_pre[chip][str(case)]['dist_post']
        ax_pre_post.scatter(dist_pre, dist_post, alpha=0.3, label=f'{condition+1}')
        ax_pre_post.set_xlabel('Pre-AOI offset')
        ax_pre_post.set_ylabel('Post-AOI offset')
        ax_pre_post.set_title(f'{chip}_{condition+1}_pre_post')
        ax_pre_post.legend()
        ax_pre_post.grid()
    
    fig_pre_post.tight_layout()
    fig_pre_post.savefig(f'{chip}_pre_post_condition.png')
    fig_pre_post.clf()

    fig.tight_layout()
    fig.savefig(f'{chip}_conditions.png')
    fig.clf()


<Figure size 1080x360 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

<Figure size 432x432 with 0 Axes>

In [64]:
# split into distance groups (15) and compute mean and variance
chips = ['R0402', 'R0603', 'R1005']
cases = range(8)
conditions = range(2)
conditions2 = range(6)

stat_data = edict()
post_pre = edict()
for chip in chips:
    stat_data[chip] = edict()
    post_pre[chip] = edict()
    for condition in conditions2:        
        data = df_chips_condition2[chip][condition].astype(float).copy()
        post_pre[chip][str(case)] = pd.DataFrame(data.iloc[:, 2:4], columns=['dist_pre','dist_post'])
        
        num_groups = 15
        data_range = data.iloc[:,0].max() - data.iloc[:,0].min()
        data_max_dists = [i * data_range / num_groups for i in np.arange(num_groups)]
        stat_data[chip][str(condition)] = pd.DataFrame(columns=['dist_mean', 'angle_mean', 'angle_stddev'])
        for i, (data_max) in enumerate(data_max_dists[1:]):
            curr_i, next_i = i, i+1
            if next_i == num_groups:
                next_i = num_groups-1
            current_distances = data.iloc[:,0]
            data_valid_df = current_distances[(current_distances < data_max_dists[next_i]) & (current_distances > data_max_dists[curr_i])]
            data_valid_idx = data_valid_df.index
            data_group_df = data.iloc[data_valid_idx, :]
            
            if len(data_valid_idx) == 0 and curr_i != next_i:
                print('[INFO] at i:',i+1, '\t (',data_max_dists[curr_i],':',data_max_dists[next_i],'):', 'None')
                continue
            else:
                angle_mean = np.mean(data_group_df.iloc[:, 1])
                angle_stddev = np.std(data_group_df.iloc[:, 1])
                dist_median = np.mean([data_group_df.iloc[:,0].max(), data_group_df.iloc[:,0].min()])        
                if np.isnan(angle_mean) or np.isnan(angle_stddev) or np.isnan(dist_median):
                    angle_mean, angle_stddev, dist_median = 0, 0, 0
            stat_data[chip][str(condition)].loc[len(stat_data[chip][str(condition)]),:] = [dist_median, angle_mean, angle_stddev]


In [65]:
# visualize angle vs dist
colordict = ['r','b']
for j, chip in enumerate(chips):
    fig = plt.figure(figsize=(15,5))  
    fig_pre_post = plt.figure(figsize=(6, 6))
    for i, condition in enumerate(conditions2):
        ax = fig.add_subplot(len(conditions2)/2, 2,i+1)
        stat_data[chip][str(condition)].plot(ax=ax, kind='scatter',x='dist_mean', y='angle_mean', color=colordict[0], label=f'{chip}_mean')
        stat_data[chip][str(condition)].plot(ax=ax, kind='scatter',x='dist_mean', y='angle_stddev', color=colordict[1], label=f'{chip}_stddev')

        lower = [(m-s) for m,s in zip(stat_data[chip][str(condition)].iloc[:,1], stat_data[chip][str(condition)].iloc[:,2])]
        upper = [(m+s) for m,s in zip(stat_data[chip][str(condition)].iloc[:,1], stat_data[chip][str(condition)].iloc[:,2])]
        ax.fill_between(stat_data[chip][str(condition)].iloc[:,0].astype(float), lower, upper, facecolor='green', alpha=0.2)
        
        ax.set_title(f'{chip}: {PRE_OR_POST_ANGLE} angle vs. SPI-Post-AOI offset distance (condition:{condition+1})')
        if ABSOLUTE_ANGLE == True:
            ax.set_xlabel('SPI-Post-AOI offset distance in percentage of chip length (%)')
        else:
            ax.set_xlabel('SPI-Post-AOI offset distance (\u03BCm)')
        ax.set_ylabel(f'{PRE_OR_POST_ANGLE} angle (degree)')
    #     ax.set_xlim([np.min(pad_plot_data.iloc[:,0]), np.max(pad_plot_data.iloc[:,0])])
        # ax.set_ylim([0, 3.5])
        ax.legend()
        ax.grid()
        
                
        ax_pre_post = fig_pre_post.add_subplot(len(conditions2)/2, 2, i+1)    
        dist_pre = post_pre[chip][str(case)]['dist_pre']
        dist_post = post_pre[chip][str(case)]['dist_post']
        ax_pre_post.scatter(dist_pre, dist_post, alpha=0.3, label=f'{condition+1}')
        ax_pre_post.set_xlabel('Pre-AOI offset')
        ax_pre_post.set_ylabel('Post-AOI offset')
        ax_pre_post.set_title(f'{chip}_{condition+1}_pre_post')
        ax_pre_post.legend()
        ax_pre_post.grid()
    
    fig_pre_post.tight_layout()
    fig_pre_post.savefig(f'{chip}_pre_post_condition2.png')
    fig_pre_post.clf()

    fig.tight_layout()
    fig.savefig(f'{chip}_conditions2.png')
    fig.clf()


<Figure size 1080x360 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

<Figure size 432x432 with 0 Axes>